## Analisis de Sentimientos

#### Por: Mauricio Santiago Valdovinos Morales

Clasificación de textos de opinión en español utilizando una red neuronal llamada Word2Vec.

In [ ]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.models import Word2Vec
from gensim import corpora, models, similarities
from os import path,listdir
import pickle
import time

reviews_doc = "data/reviews_f.dat"
reviews_doc_stop = "data/reviews_with_stopwords.dat"

with open("data/stop_words_spanish.dat", "rb") as f:
            stop_wordsE = pickle.load(f)

In [ ]:
def save_data(file,dic):
    with open(file, "wb") as f:
        pickle.dump(dic, f)

def load_data(file):
    with open(file, "rb") as f:
            return pickle.load(f)
        
def remove_stop_words(corpus):
    results = []
    for text in corpus:
        sentence = []
        for p in text:
            if p not in stop_wordsE:
                sentence.append(p)
        results.append(sentence)
    
    return results

def parser_review(path):
    file = open(path).read()
    dic = {'review':"",'review_clean':[],'sentiment': ""}
    
    #Rank
    div = file.split("rank=\"")
    rank = div[1][0]

    #Review
    div = file.split("<body>")
    div = div[1].split("</body>")
    dic['review'] = div[0]
    
    #Sentiment
    if rank == '1' or rank == '2':
        dic['sentiment'] = 'negative'
    elif rank == '3':
        dic['sentiment'] = 'neutral'
    else:
        dic['sentiment'] = 'positive'
    
    return dic

def list_reviews_features(dic):
    list_corp = []
    list_ranks = []
    list_corp_clean = []
    for r in range(len(dic)):
        list_corp.append(dic[r]['review'])
        list_ranks.append(dic[r]['sentiment'])
        list_corp_clean.append(dic[r]['review_clean'])
    return list_corp, list_ranks, list_corp_clean

def w2v_input(revs):
    w2v_int = []
    for r in revs:
        w2v_int.append(r['review_clean'])
    return w2v_int
    
def create_sentiment_dic(vocabulary):
    sentiments = {}
    for word in vocabulary:
        sentiments[word] = []
    return sentiments

def def_sentiments_words(vocabulary,reviews):
    dic = create_sentiment_dic(vocabulary)
    for doc in reviews:
        word_sentiment = []
        for rc in doc['review_clean']:
            for sentence in rc:
                for word in vocabulary:
                    if word in sentence:
                        if len(dic[word]) < 4 and doc['sentiment'] not in dic[word]:
                            dic[word].append(doc['sentiment'])
    return dic

## Generando archivo con diccionarios de la reviews

El diccionario tiene los siguientes campos:
 - Rank: Numero de estrellas de la review 1-5
 - Title: Titulo de la review
 - Summaty: pequeño resumen
 - Review: Critica
 - review_clean: review tokenizada y filtrada lista para el modelo Word2Vec

In [ ]:
def generate_reviews_doc():
    docs = []
    for f in sorted(listdir('Corpora')):
        if f[len(f)-3:] == "xml":
            f = path.join('Corpora',f)
            docs.append(parser_review(f))
    for review in docs:
        review['review_clean'] = word_tokenize(review['review'])
        for i in range(len(review['review_clean'])):
            review['review_clean'][i]  = review['review_clean'][i].lower()
        review['review_clean'] = [word for word in review['review_clean'] if word not in stop_wordsE]
    save_data(reviews_doc,docs)

In [ ]:
generate_reviews_doc()

In [ ]:
reviews = load_data(reviews_doc)
len(reviews)

## Definiendo lista de entrenamiento y prueba

In [ ]:
point = int(len(reviews) * (4/5))
train = reviews[:point]
test = reviews[point:]
print("Entrenamiento: ",len(train),"Prueba: ",len(test))

In [ ]:
#Uniendo todas las oraciones en un vector para la entrada al modelo w2v
w2v_in_train = w2v_input(train)
w2v_in_test = w2v_input(test)

## Entrenando el modelo word2vec

In [ ]:
ts = time.time()
w2v_model = Word2Vec(w2v_in_train+w2v_in_test, size=200, min_count=5, window=5, workers=4, sg=1)
w2v_model.train(w2v_in_train,total_examples=len(w2v_in_train),epochs=1000)
te = time.time()
print("time: %.4fs"%(te-ts))

Guardando el modelo Word2Vec

In [ ]:
w2v_model.save('data/model_1000.bin')

Cargando el modelo guardado

In [ ]:
w2v_model = Word2Vec.load('data/model_2.bin')

Definiendo vocabulario entregado por w2v

In [ ]:
vocabulary = list(w2v_model.wv.vocab)
print(len(vocabulary))

In [ ]:
vocabulary[15]

In [ ]:
print(w2v_model.wv.distance('película','cine'))

In [ ]:
w2v_model.wv.most_similar('obra')

#### Diccionario de las palabras

Diccionario con el indice de la palabra en el vocabulario

In [24]:
def vocab_dictionary(vocab):
    dic_words = {}
    i = 0
    for w in vocab:
        dic_words[w] = i
        i+=1
    return dic_words

In [25]:
dic_words = vocab_dictionary(vocabulary)
print(len(dic_words))

20386


#### Sentimientos a nivel de palabras

Diccionario con la palabra y una lista de los sentimientos que pueden ser positive, negative, neutral <br>

'word': ['positive', 'negative', 'neutral']

In [ ]:
sentiments = def_sentiments_words(vocabulary,reviews)
print(len(sentiments))

Guardando el diccionario de sentimientos de las palabras

In [ ]:
save_data('data/sentiments_dic.dat',sentiments)

Cargando el diccionario de sentimientos de las palabras

In [22]:
sentiments = load_data('data/sentiments_dic.dat')
print(len(sentiments))

20386


In [26]:
word = vocabulary[dic_words['peliculas']]
print(word)
print(w2v_model[word])

peliculas
[-0.05391616  0.12511624 -0.16612823 -0.02859952  0.07299291  0.11914804
  0.13209113 -0.01050776  0.01666115  0.04777921  0.1631299   0.07601571
 -0.06311471 -0.13288184 -0.19903503  0.10936841  0.1797522   0.22867373
 -0.12426182  0.01441558  0.01523442  0.00570659 -0.00233062 -0.1414325
 -0.15104525  0.07186671  0.0757286  -0.02953957 -0.04290059 -0.17482334
  0.10332102  0.2240107  -0.09068088 -0.03432599 -0.05907094 -0.41488245
  0.27800772  0.26515716  0.02702451 -0.01597686  0.08362544 -0.02558657
  0.24726865 -0.0059751   0.18630505 -0.20281783 -0.00091973  0.07688932
 -0.00815553  0.03263938  0.24263933  0.00329049 -0.03962688  0.06136565
 -0.01439736 -0.00555377  0.2341886  -0.0706628  -0.04145671  0.19139755
 -0.04085562 -0.00437373 -0.1018905  -0.06476152 -0.08691886 -0.19498949
 -0.11212474 -0.14006919 -0.14170288  0.08097848 -0.03922698 -0.02299304
  0.14234547  0.11000782 -0.0460578   0.07204275 -0.0526863  -0.07031221
  0.00768807 -0.05034762 -0.15708259  0.12

C:\Users\santiago\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


## Probando modelo

In [27]:
def test_w2v(test_list,w2v,vocab,sentiments_list):
    prediction_count = 0
    for t in test_list:
        sentiment = word_prediction(w2v,t['review_clean'],vocab,sentiments_list)
        #print("Sentiment prediction: ",sentiment," real sentiment: ",t['sentiment'])
        if sentiment == t['sentiment']:
            prediction_count+=1
    accuracy = prediction_count/len(test)*100
    print("Accuracy = ",accuracy,"% of ",len(test)," reviews")
    
def test_w2v_D(test_list,train_list,w2v,vocab,sentiments_list):
    prediction_count = 0
    for t in test_list:
        sentiment = KNN_documents(t,train_list,w2v)
        #print("Sentiment prediction: ",sentiment," real sentiment: ",t['sentiment'])
        if sentiment == t['sentiment']:
            prediction_count+=1
    accuracy = prediction_count/len(test)*100
    print("Accuracy = ",accuracy,"% of ",len(test)," reviews")

def prediction(critic,w2v,vocab,sentiments_list):
    sentiment = word_prediction(w2v,critic,vocab,sentiments_list)
    print(sentiment)

def word_prediction(w2v,sentence,vocab,sentiments_list):
    words_sentiments = []
    for word in sentence:
        if word in vocab:
            KNN_words(w2v,word,sentiments_list,words_sentiments)
    #print(words_sentiments)      
    return classify_words(words_sentiments)

def KNN_words(w2v,word,sentiments_list,words_sentiments):
    K = 5
    neighbors = w2v.wv.most_similar(word)
    for neighbor in neighbors[:K]:
        words_sentiments.append(sentiments_list[neighbor[0]])

def KNN_documents(rev,corpus,w2v):
    k = 10
    distances = []
    i=0
    for c in corpus:
        d = []
        d.append(c['sentiment'])
        d.append(w2v.wmdistance(rev,c['review_clean']))
        distances.append(d)
        print(i,end="\r")
        i+=1
    distances.sort(key=lambda x:x[1])
    neighbors_sentiments = [s[0] for s in distances[:k]]
    return classify_sentence(neighbors_sentiments)

    
def classify_words(sentiments_prediction):
    pos = 0
    neg = 0
    cont = 0
    for s in sentiments_prediction:
        if 'positive' in s:
            pos+=1
        if 'negative' in s:
            neg+=1
        cont+= len(s)
    neu = cont - (pos+neg)
    if pos > neg and pos > neu or pos == neu:
        return 'positive'
    if neg > pos and neg > neu or neg == neu:
        return 'negative'
    return 'neutral'

def classify_sentence(sentiments_prediction):
    pos = 0
    neg = 0
    cont = 0
    for s in sentiments_prediction:
        if 'positive' in s:
            pos+=1
        if 'negative' in s:
            neg+=1
    neu = len(sentiments_prediction) - (pos+neg)
    #print("Pos: ",pos, "Neg:", neg, "Neu: ",neu)
    if pos > neg and pos > neu or pos == neu:
        return 'positive'
    if neg > pos and neg > neu or neg == neu:
        return 'negative'
    return 'neutral'

In [28]:
start = time.time()
test_w2v(test,w2v_model,vocabulary,sentiments)
end = time.time()
print("time: %.4fs"%(end-start))

C:\Users\santiago\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Accuracy =  48.840206185567006 % of  776  reviews
time: 491.9397s


In [ ]:
start = time.time()
test_w2v_D(test,train,w2v_model,vocabulary,sentiments)
end = time.time()
print("time: %.4fs"%(end-start))

In [ ]:
critic = "Un spaguetti muy menor, mal rodado y peor interpretado. Dejando que un personaje mitico dentro de este subgenero como es \\\"Sabata\\\" sea visto de una manera tan espantosa."

In [ ]:
def prepare_critic(critic):
    critic = word_tokenize(critic)
    critic = [word.lower() for word in critic if word not in stop_wordsE]
    return critic
    

In [ ]:
critic = prepare_critic(critic)
print(critic)

In [ ]:
prediction(critic,w2v_model,vocabulary,sentiments)